# Populate Reference Data from Version History

**Purpose**: Extract vendor, data stream, and study information from tsDTA Version History sheets and populate reference tables.

**Input**: `md_dta_excel_file_raw` table (contains extracted data_stream_type, data_provider_name, and trial_id)

**Output**: 
- `gold_md.md_vendor` - Vendor registry with new vendors from tsDTA files
- `gold_md.md_data_stream` - Data stream registry with new stream types from tsDTA files
- `gold_md.md_study` - Study registry with new studies from tsDTA files (trial_id = study_id)

**Processing**:
- Extracts distinct vendor names, data stream types, and trial_ids from recently processed Excel files
- Uses MERGE to insert only new values (avoids duplicates)
- Auto-generates unique IDs for new entries (except study_id which uses trial_id)
- Skips 'UNKNOWN' values from extraction failures


In [ ]:
# Cell 1: Imports
import json
import uuid
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
gold_schema = globals_dict['gold_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Gold Schema: {gold_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Run ID: {databricks_run_id}")


In [ ]:
# Cell 3: Define Table Names
excel_raw_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
vendor_table = f"{catalog}.{gold_schema}.md_vendor"
data_stream_table = f"{catalog}.{gold_schema}.md_data_stream"
study_table = f"{catalog}.{gold_schema}.md_study"

print(f"Source table: {excel_raw_table}")
print(f"Vendor table: {vendor_table}")
print(f"Data stream table: {data_stream_table}")
print(f"Study table: {study_table}")


In [ ]:
# Cell 3b: Create Reference Tables if Not Exists
# Ensures tables exist before attempting MERGE operations
# Tables include 'aliases' column for entity normalization (token overlap matching)

# Create md_vendor table if it doesn't exist
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {vendor_table} (
        vendor_id STRING NOT NULL COMMENT 'Unique vendor identifier',
        vendor_name STRING NOT NULL COMMENT 'Canonical vendor name (shortest/normalized)',
        aliases ARRAY<STRING> COMMENT 'All variant names mapped to this vendor',
        vendor_description STRING COMMENT 'Vendor description',
        is_active BOOLEAN COMMENT 'Whether vendor is active',
        created_ts TIMESTAMP COMMENT 'Creation timestamp'
    ) USING DELTA
    COMMENT 'Vendor registry for data providers - uses token overlap for entity resolution'
""")
print(f"Ensured table exists: {vendor_table}")

# Add aliases column if table already exists but doesn't have it
try:
    spark.sql(f"ALTER TABLE {vendor_table} ADD COLUMN IF NOT EXISTS aliases ARRAY<STRING> COMMENT 'All variant names mapped to this vendor'")
    print(f"  Ensured aliases column exists in {vendor_table}")
except Exception as e:
    print(f"  Note: Could not add aliases column (may already exist): {e}")

# Create md_data_stream table if it doesn't exist
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {data_stream_table} (
        data_stream_id STRING NOT NULL COMMENT 'Unique data stream identifier',
        data_stream_name STRING NOT NULL COMMENT 'Canonical data stream name (shortest/normalized)',
        aliases ARRAY<STRING> COMMENT 'All variant names mapped to this data stream',
        data_stream_description STRING COMMENT 'Data stream description',
        is_active BOOLEAN COMMENT 'Whether data stream type is active',
        created_ts TIMESTAMP COMMENT 'Creation timestamp'
    ) USING DELTA
    COMMENT 'Data stream type registry - uses token overlap for entity resolution'
""")

# Add aliases column if table already exists but doesn't have it
try:
    spark.sql(f"ALTER TABLE {data_stream_table} ADD COLUMN IF NOT EXISTS aliases ARRAY<STRING> COMMENT 'All variant names mapped to this data stream'")
    print(f"  Ensured aliases column exists in {data_stream_table}")
except Exception as e:
    print(f"  Note: Could not add aliases column (may already exist): {e}")
print(f"Ensured table exists: {data_stream_table}")

# Create md_study table if it doesn't exist (trial_id = study_id)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {study_table} (
        study_id STRING NOT NULL COMMENT 'Unique study identifier (same as trial_id)',
        study_title STRING COMMENT 'Study title/name',
        study_description STRING COMMENT 'Study description',
        status STRING COMMENT 'Study status: IN_PROGRESS, PARSING_IN_PROGRESS, PARSING_COMPLETED, PARSING_FAILED, COMPLETED',
        protocol_id STRING COMMENT 'Associated protocol document ID',
        created_by_principal STRING COMMENT 'User who created the study',
        created_ts TIMESTAMP COMMENT 'Creation timestamp',
        last_updated_by_principal STRING COMMENT 'User who last updated the study',
        last_updated_ts TIMESTAMP COMMENT 'Last update timestamp',
        databricks_job_id STRING COMMENT 'Job ID for lineage',
        databricks_job_name STRING COMMENT 'Job name for lineage',
        databricks_run_id STRING COMMENT 'Run ID for lineage'
    ) USING DELTA
    COMMENT 'Study registry for clinical data management - auto-populated from tsDTA Version History'
""")
print(f"Ensured table exists: {study_table}")


In [ ]:
# Cell 4: Extract Distinct Vendors from Excel Raw Table
# Filter out UNKNOWN values and get distinct vendor names
df_vendors = spark.sql(f"""
    SELECT DISTINCT 
        data_provider_name as vendor_name
    FROM {excel_raw_table}
    WHERE data_provider_name IS NOT NULL 
      AND data_provider_name != 'UNKNOWN'
      AND TRIM(data_provider_name) != ''
""")

vendor_count = df_vendors.count()
print(f"Found {vendor_count} distinct vendor(s) to process")
df_vendors.show(truncate=False)


In [ ]:
# Cell 5: Extract Distinct Data Streams from Excel Raw Table
# Filter out UNKNOWN values and get distinct data stream types
df_streams = spark.sql(f"""
    SELECT DISTINCT 
        data_stream_type as data_stream_name
    FROM {excel_raw_table}
    WHERE data_stream_type IS NOT NULL 
      AND data_stream_type != 'UNKNOWN'
      AND TRIM(data_stream_type) != ''
""")

stream_count = df_streams.count()
print(f"Found {stream_count} distinct data stream(s) to process")
df_streams.show(truncate=False)


In [ ]:
# Cell 5b: Extract Distinct Studies (Trial IDs) from Excel Raw Table
# trial_id is synonymous with study_id
df_studies = spark.sql(f"""
    SELECT DISTINCT 
        trial_id as study_id
    FROM {excel_raw_table}
    WHERE trial_id IS NOT NULL 
      AND trial_id != 'UNKNOWN'
      AND TRIM(trial_id) != ''
""")

study_count = df_studies.count()
print(f"Found {study_count} distinct study/trial(s) to process")
df_studies.show(truncate=False)


In [ ]:
# Cell 6: Merge New Vendors into md_vendor
# Also checks if name matches any existing aliases (for entity resolution)
if vendor_count > 0:
    # Collect vendor names to process
    vendor_names = [row.vendor_name for row in df_vendors.collect()]
    
    inserted_vendors = 0
    updated_aliases = 0
    
    for vendor_name in vendor_names:
        # Generate unique vendor ID
        vendor_id = f"V-{str(uuid.uuid4())[:8].upper()}"
        
        # Escape single quotes in vendor name for SQL
        vendor_name_escaped = vendor_name.replace("'", "''")
        
        # Use MERGE to:
        # 1. Insert if vendor_name doesn't exist AND not in any aliases
        # 2. Update aliases if name matches existing vendor or is in aliases
        merge_sql = f"""
            MERGE INTO {vendor_table} AS target
            USING (SELECT '{vendor_name_escaped}' AS vendor_name) AS source
            ON UPPER(target.vendor_name) = UPPER(source.vendor_name)
               OR array_contains(target.aliases, source.vendor_name)
            WHEN MATCHED THEN
                UPDATE SET 
                    aliases = array_distinct(array_union(COALESCE(target.aliases, array()), array(source.vendor_name)))
            WHEN NOT MATCHED THEN
                INSERT (vendor_id, vendor_name, aliases, vendor_description, is_active, created_ts)
                VALUES (
                    '{vendor_id}',
                    '{vendor_name_escaped}',
                    array('{vendor_name_escaped}'),
                    'Auto-populated from tsDTA Version History',
                    true,
                    current_timestamp()
                )
        """
        result = spark.sql(merge_sql)
        
        print(f"Processed vendor: {vendor_name} (ID: {vendor_id})")
        inserted_vendors += 1
    
    print(f"\nCompleted processing {inserted_vendors} vendor(s)")
else:
    print("No new vendors to process")


In [ ]:
# Cell 7: Merge New Data Streams into md_data_stream
# Also checks if name matches any existing aliases (for entity resolution)
if stream_count > 0:
    # Collect stream names to process
    stream_names = [row.data_stream_name for row in df_streams.collect()]
    
    inserted_streams = 0
    
    for stream_name in stream_names:
        # Generate unique data stream ID
        stream_id = f"DS-{str(uuid.uuid4())[:8].upper()}"
        
        # Escape single quotes in stream name for SQL
        stream_name_escaped = stream_name.replace("'", "''")
        
        # Use MERGE to:
        # 1. Insert if data_stream_name doesn't exist AND not in any aliases
        # 2. Update aliases if name matches existing stream or is in aliases
        merge_sql = f"""
            MERGE INTO {data_stream_table} AS target
            USING (SELECT '{stream_name_escaped}' AS data_stream_name) AS source
            ON UPPER(target.data_stream_name) = UPPER(source.data_stream_name)
               OR array_contains(target.aliases, source.data_stream_name)
            WHEN MATCHED THEN
                UPDATE SET 
                    aliases = array_distinct(array_union(COALESCE(target.aliases, array()), array(source.data_stream_name)))
            WHEN NOT MATCHED THEN
                INSERT (data_stream_id, data_stream_name, aliases, data_stream_description, is_active, created_ts)
                VALUES (
                    '{stream_id}',
                    '{stream_name_escaped}',
                    array('{stream_name_escaped}'),
                    'Auto-populated from tsDTA Version History',
                    true,
                    current_timestamp()
                )
        """
        result = spark.sql(merge_sql)
        
        print(f"Processed data stream: {stream_name} (ID: {stream_id})")
        inserted_streams += 1
    
    print(f"\nCompleted processing {inserted_streams} data stream(s)")
else:
    print("No new data streams to process")


In [ ]:
# Cell 7b: Merge New Studies into md_study
# Creates study records for trial_ids found in tsDTA files
if study_count > 0:
    # Collect study_ids to process
    study_ids = [row.study_id for row in df_studies.collect()]
    
    inserted_studies = 0
    for study_id_val in study_ids:
        # Escape single quotes in study_id for SQL
        study_id_escaped = study_id_val.replace("'", "''")
        
        # Use MERGE to insert only if study_id doesn't exist
        merge_sql = f"""
            MERGE INTO {study_table} AS target
            USING (SELECT '{study_id_escaped}' AS study_id) AS source
            ON target.study_id = source.study_id
            WHEN NOT MATCHED THEN
                INSERT (study_id, study_title, study_description, status, protocol_id, 
                        created_by_principal, created_ts, last_updated_by_principal, last_updated_ts,
                        databricks_job_id, databricks_job_name, databricks_run_id)
                VALUES (
                    '{study_id_escaped}',
                    'Auto-created from tsDTA: {study_id_escaped}',
                    'Study auto-populated from tsDTA Version History during import',
                    'IN_PROGRESS',
                    NULL,
                    '{created_by_principal}',
                    current_timestamp(),
                    '{created_by_principal}',
                    current_timestamp(),
                    '{databricks_job_id}',
                    'job_cds_tsdta_xls_processor',
                    '{databricks_run_id}'
                )
        """
        result = spark.sql(merge_sql)
        
        print(f"Processed study: {study_id_val}")
        inserted_studies += 1
    
    print(f"\nCompleted processing {inserted_studies} study/trial(s)")
else:
    print("No new studies to process")


In [ ]:
# Cell 8: Verification - Show Current Reference Data
print("=" * 60)
print("Current Studies in md_study:")
print("=" * 60)
spark.sql(f"SELECT study_id, study_title, status, created_ts FROM {study_table} ORDER BY created_ts DESC").show(truncate=False)

print("=" * 60)
print("Current Vendors in md_vendor:")
print("=" * 60)
spark.sql(f"SELECT vendor_id, vendor_name, is_active, created_ts FROM {vendor_table} ORDER BY created_ts DESC").show(truncate=False)

print("=" * 60)
print("Current Data Streams in md_data_stream:")
print("=" * 60)
spark.sql(f"SELECT data_stream_id, data_stream_name, is_active, created_ts FROM {data_stream_table} ORDER BY created_ts DESC").show(truncate=False)


In [ ]:
# Cell 9: Summary
print("\n" + "=" * 60)
print("REFERENCE DATA POPULATION COMPLETE")
print("=" * 60)
print(f"Studies processed: {study_count}")
print(f"Vendors processed: {vendor_count}")
print(f"Data streams processed: {stream_count}")
print(f"Job ID: {databricks_job_id}")
print(f"Run ID: {databricks_run_id}")
